# Main code

Using Python 3.11.8

### Master dependencies

In [ ]:
%pip install torch
%pip install torchvision
%pip install matplotlib
%pip install split-folders
%pip install pipreqs
%pip install subprocess
%pip install scikit-learn
%pip install lime
%pip install opencv-python

### Saving dependencies to requirements.txt

In [ ]:
import subprocess

installed_packages = subprocess.check_output(['pip', 'freeze']).decode('utf-8').strip().split('\n')
local_packages = [pkg for pkg in installed_packages if pkg.startswith('-e') or '==' in pkg]

with open('requirements.txt', 'w') as file:
    for pkg in local_packages:
        file.write(pkg + '\n')

print("Requirements saved to requirements.txt")

### Installing Dependencies

In [ ]:
%pip install -r requirements.txt

### Importing dependencies

In [ ]:
import os
from PIL import Image
from lime import lime_image
from skimage.segmentation import mark_boundaries
import torch.nn.functional as F

import torch
import splitfolders 
import matplotlib.pyplot as plt
from torchvision.transforms import Normalize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import Resize
from torchvision.datasets import ImageFolder
from torchvision.transforms import RandomVerticalFlip
from torchvision.transforms import RandomHorizontalFlip

## Loading and Pre Processing Data

Resizing the images (so they each have the same amount of pixels, not to overfit to larger images) and converting them to Tensor, to find the mean and standard deviation of each channel (R, G and B) of the images. This will be used to normalize them.

In [ ]:
root = '.'

In [ ]:
transform = Compose([Resize(size=(64, 64)),ToTensor()])

dataset = ImageFolder(root=f"{root}/segdata", transform=transform)

means = torch.zeros(3)
stds = torch.zeros(3)
num_pixels = 0

for image, _ in dataset:
    means += torch.mean(image, dim=(1, 2))
    stds += torch.std(image, dim=(1, 2))
    num_pixels += image.size(1) * image.size(2)

means /= len(dataset)
stds /= len(dataset)

means = means.tolist()
stds = stds.tolist()

print("Mean:", means)
print("Standard Deviation:", stds)

Splitting the data into train, validation and test sets of size 70%, 20% and 10% respectively, before applying normalization and other data augmentation functions.

In [ ]:
splitfolders.ratio(f"{root}/segdata", output=f"{root}/splitted_data", seed=42,ratio=(.7, .2, .1), group_prefix=None,move=False)

In [ ]:
transform_train = Compose([RandomHorizontalFlip(), RandomVerticalFlip(), Resize(size=(64,64)), ToTensor(),Normalize(mean=means, std=stds)])

dataset = ImageFolder(root=f"{root}/segdata")
image = dataset[9][0]

plt.subplot(1, 2, 1)
plt.imshow(transform(image).permute(1, 2, 0))
plt.title('Original Image')

transformed_image = transform_train(image)

plt.subplot(1, 2, 2)
plt.imshow(transformed_image.permute(1, 2, 0))  # Assuming transformed_image is a torch.Tensor
plt.title('Transformed Image')

plt.show()